<a href="https://www.kaggle.com/code/sabinakhadysy/learning-equality?scriptVersionId=115762204" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv
/kaggle/input/learning-equality-curriculum-recommendations/topics.csv
/kaggle/input/learning-equality-curriculum-recommendations/correlations.csv
/kaggle/input/learning-equality-curriculum-recommendations/content.csv


# EDA

In [2]:
topics_data = pd.read_csv("../input/learning-equality-curriculum-recommendations/topics.csv")
topics_data.head()

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,NaN,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,NaN,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True


In [3]:
print(f' There are {topics_data.shape[0]} rows and {topics_data.shape[1]} columns in the topics dataframe.')

 There are 76972 rows and 9 columns in the topics dataframe.


In [4]:
#Checking how many missing data we have in topics_data
missing_topics_data = topics_data.isnull().sum()
print(f'In topics dataset we have missing values in following columns:\n{missing_topics_data[missing_topics_data > 0]}')

In topics dataset we have missing values in following columns:
title              2
description    42019
parent           171
dtype: int64


171 values are missing in the parent column in topics_data dataframe which means that we have information gathered about 171 different root nodes (channels).

In [5]:
#Checking if there are also 171 different channles in the channel column 
print(f'There are {len(topics_data.channel.unique())} unique values in the channel column.')

There are 171 unique values in the channel column.


In [6]:
#Checking if level equal 0 is present 171 times
print(f'Level equal 0 is present {topics_data[topics_data.level==0].level.count()} times in the topics dataset.')

Level equal 0 is present 171 times in the topics dataset.


In [7]:
#Checking if the indices for unique channel values are the same as the indices for level equal 0 and for missing values in the parent column.
root_topics_data = topics_data.iloc[list(topics_data.level==0 & topics_data.parent.isnull())] #part of the topics dataframe only with root topics
print(f'The indices for unique channel values are the same as the indices for level equal 0 and for missing values in the parent column as in the'\
      f'root_topics_data (part of topics dataframe only with root topics) we have {len(root_topics_data.channel.unique())} unique channel names.')

The indices for unique channel values are the same as the indices for level equal 0 and for missing values in the parent column as in theroot_topics_data (part of topics dataframe only with root topics) we have 171 unique channel names.


In [8]:
content_data = pd.read_csv('../input/learning-equality-curriculum-recommendations/content.csv')
content_data.head()

,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,NaN,es,NaN,NaN
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,NaN,it,NaN,NaN
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,NaN,es,NaN,NaN
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA


In [9]:
print(f' There are {content_data.shape[0]} rows and {content_data.shape[1]} columns in the content dataframe.')

 There are 154047 rows and 8 columns in the content dataframe.


In [10]:
correlations_data = pd.read_csv('../input/learning-equality-curriculum-recommendations/correlations.csv')
correlations_data.head()

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4


In [11]:
print(f' There are {correlations_data.shape[0]} rows and {correlations_data.shape[1]} columns in the correlations dataframe.')

 There are 61517 rows and 2 columns in the correlations dataframe.


In [12]:
sample_data = pd.read_csv('../input/learning-equality-curriculum-recommendations/sample_submission.csv')
sample_data.head()

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_4054df11a74e,c_3695c5dc1df6 c_f2d184a98231


In [13]:
print(f' There are {sample_data.shape[0]} rows and {sample_data.shape[1]} columns in the sample submission dataframe.')

 There are 5 rows and 2 columns in the sample submission dataframe.


Creating some output just for the purpose of submitting this file.

In [14]:
output_df = pd.read_csv("/kaggle/input/learning-equality-curriculum-recommendations/sample_submission.csv")


In [15]:
output_df.to_csv('submission.csv', index=False)